<a href="https://colab.research.google.com/github/shuchimishra/Model_deployment_using_Kubernetes_and_Docker/blob/main/Running_Flask_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r "/content/drive/MyDrive/Data Science & Machine Learning/MLOps/Repository/model_output" "/content"

In [3]:
!ls "/content/model_output"

sa_encoder.vocab.tokens  sentiment_analysis.hdf5


In [4]:
import socket
print(socket.gethostbyname(socket.gethostname()))

172.28.0.12


In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
import threading

app = Flask(__name__)
padding_size = 1000
model = tf.keras.models.load_model('/content/model_output/sentiment_analysis.hdf5')
text_encoder = tfds.deprecated.text.TokenTextEncoder.load_from_file("/content/model_output/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':8000}).start()

Model and Vocabalory loaded.......
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit


In [6]:
import requests
req = requests.get("http://172.28.0.12:8000/")
print(req.status_code)
print(req.text)

INFO:werkzeug:172.28.0.12 - - [18/Jun/2024 06:19:10] "GET / HTTP/1.1" 200 -


200
I am alive!


In [7]:
seclassifier = "http://172.28.0.12:8000/seclassifier"

In [10]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post(seclassifier,  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

Still working my way through it but definitely changes your view on investment. Wish it was available on Audible
1/1 [==============================] - ETA: 0s

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


1/1 [==============================] - 11s 11s/step


INFO:werkzeug:172.28.0.12 - - [18/Jun/2024 06:20:12] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[0.8370057344436646]],"sentiment ":"positive"}



In [11]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post(seclassifier,  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

I thought this will be good one. But I was wrong
1/1 [==============================] - ETA: 0s

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


1/1 [==============================] - 10s 10s/step


INFO:werkzeug:172.28.0.12 - - [18/Jun/2024 06:20:37] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[-0.8676939606666565]],"sentiment ":"Negative"}



In [12]:
!ps -eaf | grep python

root          64       8  3 06:16 ?        00:00:08 [python3] <defunct>
root          65       8  0 06:16 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root          87       8  1 06:16 ?        00:00:04 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root         252      87  8 06:16 ?        00:00:19 /usr/bin/python3 -m colab_kernel_launcher -f /ro
root         279       1  0 06:16 ?        00:00:00 /usr/bin/python3 /usr/local/lib/python3.10/dist-
root         653     650  0 06:17 ?        00:00:00 python3 /opt/google/drive/drive-filter.py
root        1537     252  0 06:20 ?        00:00:00 /bin/bash -c ps -eaf | grep python
root        1539    1537  0 06:20 ?        00:00:00 grep python


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


In [13]:
import flask
flask.__version__

'2.2.5'

In [14]:
!pip install flask-ngrok

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


In [15]:
!pip install pyngrok==4.1.1

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=e8de73b0bcbfd54e039904f62a69647152c1762ce1146fcd4ba40babd9f471af
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


In [16]:
ngrok_token = "2i2a0OR6yui4B2pyILwXM8Dx3FG_7otRNAGjTX76uJRYHmLyD" # @param {type:"string"}


In [18]:
# !cat /root/.ngrok2/ngrok.yml

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok
!ngrok authtoken `echo $ngrok_token`


app_v2 = Flask(__name__)
run_with_ngrok(app_v2)

padding_size = 1000
model = tf.keras.models.load_model('/content/model_output/sentiment_analysis.hdf5')
text_encoder = tfds.deprecated.text.TokenTextEncoder.load_from_file("/content/model_output/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app_v2.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app_v2.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

if __name__ == '__main__':
    app_v2.run()

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


Model and Vocabalory loaded.......
 * Serving Flask app '__main__'


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 497, in _make_request
    conn.request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 395, in request
    self.endheaders()
  File "/usr/lib/pyt

 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shap